
# Master Informatique, parcours Data Mining

### Carnets de note Python pour le cours de Text Mining

Julien Velcin, laboratoire ERIC, Université Lyon 2



# Prétraitements (partie 1)

## Quelques prétraitements à connaître

- expressions régulières et nettoyages simples
- segmentation en mots (*tokenization*)
- mots-outils
- stemming et lemmatisation
- n-grammes et collocations

## Quelques prétraitements à connaître

- **expressions réguières et nettoyages simples**
- segmentation en mots (*tokenization*)
- mots-outils
- stemming et lemmatisation
- n-grammes et collocations

## expressions régulières 

In [9]:
import os

texte = "" # chaîne vide
with open("Frank Herbert - Children of Dune.txt", "r", encoding='utf8') as f:
    texte = texte.join(line.rstrip("\n") + " " for line in f.readlines())

In [37]:
import re
pattern = re.compile("the")

# cherche toutes les occurrences
res = pattern.finditer(texte)

start_pattern = [m.start() for m in res]

#print(start_pattern)
print(len(start_pattern))

10566


In [32]:
import re # notez qu'importer à nouveau la librairie n'est pas nécessaire
pattern = re.compile("spice", re.IGNORECASE)

# cherche la première occurrence seulement
# res = pattern.search(texte)  

# cherche toutes les occurrences
res = pattern.finditer(texte)

#start_pattern = [m.start() for m in res]

#print(len(start_pattern))
sp = [m.span() for m in res]
print(sp)

#print(texte[start_pattern[100]-10 : start_pattern[100]+20])

res

[(720, 725), (9600, 9605), (9631, 9636), (10319, 10324), (15334, 15339), (19340, 19345), (19713, 19718), (24076, 24081), (24412, 24417), (25627, 25632), (26968, 26973), (28993, 28998), (29474, 29479), (33427, 33432), (34894, 34899), (34905, 34910), (69902, 69907), (76637, 76642), (81060, 81065), (88639, 88644), (106108, 106113), (111491, 111496), (113127, 113132), (115740, 115745), (116146, 116151), (119841, 119846), (135605, 135610), (136136, 136141), (136495, 136500), (137107, 137112), (141280, 141285), (146234, 146239), (150460, 150465), (164675, 164680), (166841, 166846), (170651, 170656), (177143, 177148), (189222, 189227), (198956, 198961), (218348, 218353), (225894, 225899), (235596, 235601), (237125, 237130), (260375, 260380), (263518, 263523), (290355, 290360), (291127, 291132), (291999, 292004), (297965, 297970), (299531, 299536), (299651, 299656), (304082, 304087), (323664, 323669), (347268, 347273), (347287, 347292), (348333, 348338), (352391, 352396), (355282, 355287), (35

Si on souhaite trouver toutes les occurrences, on peut afficher ce qu'on appelle un concordancier.

Depuis le XIIIème siècle, un **concordancier** est une liste triée alphabétiquement des principaux mots employés dans un corpus, précisant **chaque instance** des mots accompagnée de leur **contexte immédiat**. 

<img src="img/concordance.jpg" style='height: 400px'/>

*Cruden's Concordance (concordance of the King James Bible that was single-handedly created by Alexander Cruden)*

In [34]:
import pandas as pd

window = 50

def concord(texte, pat):
    pattern = re.compile(pat)
    res = pattern.finditer(texte)
    pos_pattern = [m.span() for m in res]
    context_left = pd.DataFrame([texte[i-window:i-1] for (i, j) in pos_pattern])
    center = pd.DataFrame([texte[i: j] for (i, j) in pos_pattern])
    context_right = pd.DataFrame([texte[j+1:j+window] for (i, j) in pos_pattern])
    return (pd.concat([context_left, center, context_right], axis=1))

Quelques exemples d'expressions régulières :

In [44]:
# suite exacte :
#concord(texte, "dune")

# présence optionnelle d'un caractère :
concord(texte, "dune?")

# chiffre :
#concord(texte, "[0-9]")

# suite de chiffres :
#concord(texte, "[0-9]+")   # le + indique qu'il doit au moins y avoir un chiffre
#concord(texte, "\d+")   # \d est plus général pour les suites de chiffre

# recherche de deux motifs :
#concord(texte, "Paul|Atreid")    # le | (ou "pipe") indique un OU logique

,0,0,0
0,"round the Lady Jessica, reaching far out into the",dun,flatness of the landing plain upon which her tran
1,"arachans -- those high, crescent-shaped migratory",dune,which moved like waves around Arrakis. This was
2,"rrakis. This was Kedem, the inner desert, and its",dune,were rarely marked these days by the irregularit
3,m's progress. Sunset drew bloody streaks over the,dune,", imparting a fiery light to the shadow edges. A"
4,and green. A diamond-shaped oasis of planted,dune,"spread beneath his high perch, focusing his atte"
...,...,...,...
75,"fficult to take his gaze away from the sands, the",dune,-- the great emptiness. Here at the edge of the
76,"e dust, the sparse and lonely plants and animals,",dune,"merging into dune, desert into desert. Behind"
77,"and lonely plants and animals, dune merging into",dune,desert into desert. Behind him came the soun
78,him. Now Stilgar and Irulan waited in the Temple,dun,"eons. Perhaps they would die, but there might be"


L'action la plus élémentaire consiste à nettoyer le texte s'il contient des symboles non souhaités (par ex. des caractères unicodes).

In [31]:
import re

txt_a_nettoyer = "ÅVoilà un texte qui pose problͰme  ̶ voyons ce que l ̕on peut y faire"

txt_propre = re.sub("Å", " ", txt_a_nettoyer)
txt_propre = re.sub("Ͱ", "è", txt_propre)
txt_propre = re.sub(" ̶", "-", txt_propre)
txt_propre = re.sub(" ̕", "'", txt_propre)

print(txt_propre)

 Voilà un texte qui pose problème - voyons ce que l'on peut y faire


In [32]:
clean_unicode = {
  "Å": " ",
  "Ͱ": "è",
  " ̶" : "-",
  " ̕" : "'"
}

txt_propre_2 = txt_a_nettoyer
for c in clean_unicode:
    txt_propre_2 = re.sub(c, clean_unicode[c], txt_propre_2)
    
print(txt_propre_2)

 Voilà un texte qui pose problème - voyons ce que l'on peut y faire


Il est également simple de remplacer une expression par une autre, par exemple les occurrences de "chaumas" par "poison".

In [33]:
# la fonction re.sub permet de faire un "remplacer tout" pour les occurrences d'un motif donné
# attention à penser à sauvegarder le résultats, c'est-à-dire la nouvelle chaîne de caractères

texte_comp = re.sub("chaumas", "poison", texte)

In [34]:
# on vérifie que le remplacement a bien été réalisé

num_it = 0 # numéro de l'instance qu'on souhaite vérifier

liste_it_texteini = [m.start() for m in re.finditer("chaumas", texte)]
print(texte[liste_it_texteini[num_it]-10: liste_it_texteini[num_it]+50])

liste_it_textetransf = [m.start() for m in re.finditer("poison", texte_comp)]
print(texte_comp[liste_it_textetransf[num_it]-10: liste_it_textetransf[num_it]+50])
#print(sp_pgpp_comp)

ill it be chaumas--poison in the food?     He shook his head
a drop of poison on its tip. Ah-ah! Don't pull away or you'l


On peut utiliser des expressions régulières plus complexes.

Par exemple pour chercher tous les nombres qui peuvent comporter des espaces, des virgules ou des points, dans leur écriture :

In [35]:
int_pat = "[0-9]+((\s|,|\.)|[0-9])+"   #\s indique n'importe quel espace

concord(texte, int_pat)

,0,0,0
0,,1965,ook 1 DUNE = = = = = = A beginning is the time
1,,1,UNE = = = = = = A beginning is the time for ta
2,that you first place him in his time: born in the,57,"h year of the Padishah Emperor, Shaddam IV. And t"
3,"= = YUEH (yu'e), Wellington (weling-tun), Stdrd","10,082","10,191; medical doctor of the Suk School (grd Std"
4,"UEH (yu'e), Wellington (weling-tun), Stdrd 10,082","10,191","medical doctor of the Suk School (grd Stdrd 10,1"
...,...,...,...
81,rice on the Imperial market has ranged as high as,"620,000",olaris the decagram. MENTAT: that class of Imperi
82,quartz. Noted for extreme tensile strength (about,"450,000",ilos per square centimeter at two centimeters' th
83,orms grow to enormous size (specimens longer than,400,eters have been seen in the deep desert) and live
84,e-dimensional image from a solido projector using,360,degree reference signals imprinted on a shigawire


Un nettoyage standard pour les analyses ultérieures consiste à passer tout le corpus en minuscule. C'est une forme de normalisation permettant de rapprocher des termes comme "Unité" et "unité" par exemple.

In [36]:
print("avant : ", texte[1500:1600])
print("après : ", texte[1500:1600].lower())

texte_minuscules = texte.lower()

avant :  e glittering jewels.     "Is he not small for his age, Jessica?" the old woman asked. Her voice whee
après :  e glittering jewels.     "is he not small for his age, jessica?" the old woman asked. her voice whee


## Quelques prétraitements à connaître

- expressions réguières et nettoyages simples
- **segmentation en mots (*tokenization*)**
- mots-outils
- stemming et lemmatisation
- n-grammes et collocations

## segmentation en mots (*tokenization*)

Split a string into basic (lexical) **units**, such as words.

Different rules for different languages:

我已经等我的包裹三星期了！
<br/>
(I've been waiting three weeks for my parcel to arrive!)

En français,c'est assez simple mais<br/>ilfaut faire attention<br/>notamment sur plusieurs lignes.

Even in English or French, you can have difficulties splitting strings.

In [37]:
chaine = "Here is, for sure, a small paragraph I'd like to parse\non multiple lines.Is this that simple?I guess not."
re.sub("[^(\s|\w)]", "", chaine)

'Here is for sure a small paragraph Id like to parse\non multiple linesIs this that simpleI guess not'

En général, avec la plupart des langues européennes, c'est une tâche assez facile si on utilise les séparateurs suivants :

, . \n \t - : ; ( ) ! ? [ ] _ ' "  (etc.)

Mais... :

>"Harry Potter" => "Harry", "Potter"<br/>
>"rez-de-chaussée" => "chaussée", "de", "rez"<br/>
>"idiot?" => "idiot"<br/>
>"C.E.O" => "C", "E", "O"

Nous allons voir dans les carnets qui suivant et que les librairies Python comme *nltk*, *scikit-learn* ou *spacy* embarquent des tokeniseurs puissants.

Cependant, essayons de réaliser cette opération manuellement afin de mesurer les mécanismes sous-jacents avec le texte suivant :

"The challenge of exploiting the large proportion of enterprise information that originates in "unstructured" form has been recognized for decades.[7] It is recognized in the earliest definition of business intelligence (BI), in an October 1958 IBM Journal article by H.P. Luhn, A Business Intelligence System, which describes a system that will:"<br/>(excerpt of https://en.wikipedia.org/wiki/Text_mining)

In [38]:
ch = "The challenge of exploiting the large proportion of enterprise information that originates in \"unstructured\" form has been recognized for decades.[7] It is recognized in the earliest definition of business intelligence (BI), in an October 1958 IBM Journal article by H.P. Luhn, A Business Intelligence System, which describes a system that will:"

re.split(r'\W+', ch)

['The',
 'challenge',
 'of',
 'exploiting',
 'the',
 'large',
 'proportion',
 'of',
 'enterprise',
 'information',
 'that',
 'originates',
 'in',
 'unstructured',
 'form',
 'has',
 'been',
 'recognized',
 'for',
 'decades',
 '7',
 'It',
 'is',
 'recognized',
 'in',
 'the',
 'earliest',
 'definition',
 'of',
 'business',
 'intelligence',
 'BI',
 'in',
 'an',
 'October',
 '1958',
 'IBM',
 'Journal',
 'article',
 'by',
 'H',
 'P',
 'Luhn',
 'A',
 'Business',
 'Intelligence',
 'System',
 'which',
 'describes',
 'a',
 'system',
 'that',
 'will',
 '']

## Quelques prétraitements à connaître

- expressions réguières et nettoyages simples
- segmentation en mots (*tokenization*)
- **mots-outils**
- stemming et lemmatisation
- n-grammes et collocations

## mots-outils

Commençons par quelques observations générales sur des corpus bien connus.

Mots les plus fréquents observés dans Harry Potter :

<img src="img/zipf_1.png" width="100%">

Cette forme particulière est appelée "loi de Zipf". Elle a été observée pour la première fois par le linguiste George K. Zipf (1902-1950).

La loi de Zipf indique que si l'on observe un corpus suffisamment grand, la fréquence d'apparition d'un mot est **inversement proportionnelle** à son rang dans la table des fréquences.

Les mots-outils sont des mots qui n'apportent pas beaucoup d'information et sont surtout utilisés pour construire les phrases. Ils jouent donc un rôle *fonctionnel*.

Il faut faire un peu attention mais, la plupart du temps, on les enlève.

La librairie *scikit-learn* fournit ses propres listes de mots-outils, mais *nltk* a une bibliothèque plus fournie de langues. Il est bien entendu possible de charger votre propre liste de mots-outils.

In [46]:
# il ne faut pas oublier de télécharger les ressources nécessaires, en particulier le module "stopwords" de "Corpora"
# import nltk
# nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [47]:
from nltk.corpus import stopwords
print("Mots-outils en anglais : {}".format(stopwords.words('english')[0:20]))
print("Mots-outils en français : {}".format(stopwords.words('french')[0:20]))
print("Mots-outils en arabe : {}".format(stopwords.words('arabic')[0:20]))

Mots-outils en anglais : ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']
Mots-outils en français : ['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur']
Mots-outils en arabe : ['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي']


On peut tester ces listes à la main en passant par un *tokenizer* maison sous nltk.

In [60]:
from nltk.tokenize import word_tokenize  

example_sent = "Here is a simple example of a sentence with multiple stopwords"
  
stop_words = set(stopwords.words('english'))  
word_tokens = word_tokenize(example_sent)  
word_tokens
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
print(filtered_sentence)

['Here', 'simple', 'example', 'sentence', 'multiple', 'stopwords']


Suivant l'hypothèse du "petit monde", on peut développer un algorithme simple mais efficace d'identification de la langue si la longueur d'un texte est assez grande.

Si $|T| > 30$, alors on obtient 99,5% de réussite (Grefenstette 1995).

In [42]:
import numpy as np

languages = (
        "arabic",
        "danish",
        "dutch",
        "english",
        "finnish",
        "french",
        "german",
        "hungarian",
        "italian",
        "norwegian",
        "portuguese",
        "romanian",
        "russian",
        "spanish",
        "swedish",
    )

ch = "Las Antillas, o islas del Caribe, estan situadas junto al tropico de Cancer. Alli hace calor todo el ano."
#ch = "Voilà un test en français un peu plus long et on continue"
#ch = "hello there, how are you guys?"

# compte le nombre de mots-outils pour une langue donnée
def nb_stopwords(s, langue):
    stop_words = set(stopwords.words(langue))
    word_tokens = set(word_tokenize(s))
    return len(stop_words.intersection(word_tokens))

list_val = [nb_stopwords(ch, l) for l in languages]

print(list_val)
print("My guess is: {}".format(languages[np.argmax(list_val)]))

[0, 1, 2, 1, 0, 1, 0, 2, 3, 1, 2, 4, 0, 6, 1]
My guess is: spanish


Même graphique mais en supprimant les mots-outils anglais :
    
<img src="img/zipf_2.png" width="100%">

## Quelques prétraitements à connaître

- expressions réguières et nettoyages simples
- segmentation en mots (*tokenization*)
- mots-outils
- **stemming et lemmatisation**
- n-grammes et collocations

## racinisation et lemmatisation

D'autres prétraitements sont également disponibles, tels que :

* racinisation (*stemming*) : trouver la racine des mots, comme dans :
     
> learn: learns, learned, learning...<br/>
> march: marcher, marchera, marcherai...
 
* lemmatisation (*lemmatization*) : trouver le lemme, comme dans :

> to be: am, are<br/>
> être : suis, sont...  

La racinisation (*stemming*) nous aide à :

- rapprocher deux documents sémantiquement liés mais n'utilisant pas exactement les mêmes termes
- réduire la taille du vocabulaire (cf. malédiction de la dimension)

Ok pour :

    adventur: [adventure,adventurer,adventurers,adventures,adventurous]

mais...

    anim: [animal, animals, animation]



In [43]:
from nltk.stem.snowball import SnowballStemmer

stemmer_fr = SnowballStemmer("french")
stemmer_en = SnowballStemmer("english") # à priori, l'algorithme de Porter

In [44]:
#stemmer.stem("maisons")
print("En français :")
print([stemmer_fr.stem(w) for w in ["marcher", "marcherons", "marcherait", "marché"]])
print([stemmer_fr.stem(w) for w in ["aventure", "aventures", "aventuriers", "aventureux"]])
print("En anglais :")
print([stemmer_en.stem(w) for w in ["adventures", "adventurers", "adventurous", "adventurer", "adventurous"]])

En français :
['march', 'march', 'march', 'march']
['aventur', 'aventur', 'aventuri', 'aventur']
En anglais :
['adventur', 'adventur', 'adventur', 'adventur', 'adventur']


In [45]:
# Pour l'utilisation de CountVectorizer avec la librairie scikit-learn, voir le carnet 2.1 :
from sklearn.feature_extraction.text import CountVectorizer

analyzer = CountVectorizer().build_analyzer()

def stemmed_words_en(doc):
    return (stemmer_en.stem(w) for w in analyzer(doc))

stem_vectorizer_en = CountVectorizer(analyzer=stemmed_words_en)

X = ["Here is the adventures of an adventurous adventurer"]
stem_vectorizer_en.fit(X)
stem_vectorizer_en.vocabulary_

{'here': 2, 'is': 3, 'the': 5, 'adventur': 0, 'of': 4, 'an': 1}

La lemmatisation (*lemmatization*) transforme les mots en leur lexème sous-jacent (*lemma*).

> running --> to run (verb) / running (noun)<br/>
> are &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;--> to be (verb)

Pour cela, il est nécessaire de résoudre le problème de détection des catégories grammaticales (*POS tagging*) et de recourir à une base de connaissances lexicales, voire des données annotées.

La lemmatisation est plus précise que la racinisation, mais elle nécessite plus de ressources. Cela revient à un choix entre + de précision et - de rappel (lemmatisation) vs. - de précision et + de rappel (racinisation).

En utilisant nltk :

In [46]:
from nltk import word_tokenize, pos_tag, ne_chunk
 
sentence = "Mark and John are working at Google."
 
print(ne_chunk(pos_tag(word_tokenize(sentence))))

(S
  (PERSON Mark/NNP)
  and/CC
  (PERSON John/NNP)
  are/VBP
  working/VBG
  at/IN
  (ORGANIZATION Google/NNP)
  ./.)


Mais également la library stanza (https://stanfordnlp.github.io/stanza/) qui fonctionne en général bien dans beaucoup de langues :

In [47]:
import stanza
#stanza.download('fr') # modèle pour le français
#   (attention, c'est assez lourd ~500Mo, il vaut mieux avoir une bonne connexion)
nlp = stanza.Pipeline('fr') # pipeline pour le français
doc = nlp("Barack Obama est né à Hawaii.") # premier test d'annotation

2021-12-17 09:39:40 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | wikiner |

2021-12-17 09:39:40 INFO: Use device: cpu
2021-12-17 09:39:40 INFO: Loading: tokenize
2021-12-17 09:39:40 INFO: Loading: mwt
2021-12-17 09:39:40 INFO: Loading: pos
2021-12-17 09:39:40 INFO: Loading: lemma
2021-12-17 09:39:40 INFO: Loading: depparse
2021-12-17 09:39:40 INFO: Loading: ner
2021-12-17 09:39:41 INFO: Done loading processors!


In [62]:
doc

[
  [
    {
      "id": 1,
      "text": "Barack",
      "lemma": "Barack",
      "upos": "PROPN",
      "head": 4,
      "deprel": "nsubj",
      "start_char": 0,
      "end_char": 6,
      "ner": "B-PER"
    },
    {
      "id": 2,
      "text": "Obama",
      "lemma": "Obama",
      "upos": "PROPN",
      "head": 1,
      "deprel": "flat:name",
      "start_char": 7,
      "end_char": 12,
      "ner": "E-PER"
    },
    {
      "id": 3,
      "text": "est",
      "lemma": "être",
      "upos": "AUX",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin",
      "head": 4,
      "deprel": "aux:tense",
      "start_char": 13,
      "end_char": 16,
      "ner": "O"
    },
    {
      "id": 4,
      "text": "né",
      "lemma": "naître",
      "upos": "VERB",
      "feats": "Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part",
      "head": 0,
      "deprel": "root",
      "start_char": 17,
      "end_char": 19,
      "ner": "O"
    },
    {
      "id": 5,
      "text": "à

Réécriture du texte avec les lemmes :

In [49]:
[w.lemma for w in doc.sentences[0].words]

['Barack', 'Obama', 'être', 'naître', 'à', 'Hawaii', '.']

On peut aller plus loin en identifiant les entités nommées, par exemple.

In [68]:
texte_rois = ""
with open(os.path.join("datasets", "rois.txt"), "r", encoding='utf8') as f:
    texte_rois = texte_rois.join(line.rstrip("\n") + " " for line in f.readlines())

# la ligne suivante permet de retirer les espaces redondantes à l'intérieur de la chaîne de caractères
#texte_rois = " ".join(texte_rois.split())    

In [70]:
doc_rois = nlp(texte_rois)

In [71]:
#Find named entities, phrases and concepts
c = 0
for entity in doc_rois.ents:
    print(entity.text, entity.type)
    if c>50:
        break;
    c +=1

MAURICE DRUON PER
Académie française ORG
ROIS MAUDITS MISC
LIVRE I  Le Roi de Fer MISC
Le Roi de fer MISC
Maurice Druon PER
Pion PER
Éditions Del Duca MISC
Edmond PER
Jules de Concourt      PROLOGUE PER
Philippe IV PER
la France LOC
Flamands MISC
Anglais MISC
Aquitaine LOC
Papauté MISC
Avignon LOC
Parlements MISC
Edouard II d’Angleterre PER
Russie LOC
Aucune LOC
Église, spolié les Juifs MISC
Trésor ORG
Les LOC
Les LOC
Les LOC
État LOC
la France LOC
Français MISC
Ordre souverain des chevaliers du Temple ORG
Cette LOC
Templiers MISC
Philippe le Bel PER
Histoire MISC
Toutes LOC
PREMIÈRE PARTIE ORG
MALÉDICTION      I    LA REINE PER
AMOUR MISC
Angleterre LOC
Isabelle PER
Guillaume d’Aquitaine PER
Ah ! MISC
Guillaume PER
Madame PER
Non PER
Monseigneur d’Artois PER
Veillez MISC
France LOC
Madame PER
Jeanne de Joinville PER
Roger Mortimer PER
Angleterre LOC


In [72]:
list_loc = [ent.text for ent in doc_rois.entities if ent.type=="LOC"]
list_loc[0:50]

['la France',
 'Aquitaine',
 'Avignon',
 'Russie',
 'Aucune',
 'Les',
 'Les',
 'Les',
 'État',
 'la France',
 'Cette',
 'Toutes',
 'Angleterre',
 'France',
 'Angleterre',
 'France',
 'Angleterre',
 'France',
 'Artois',
 'Artois',
 'Artois',
 'Angleterre',
 'Artois',
 'Navarre',
 'France',
 'France',
 'Artois',
 'France',
 'Or',
 'Damas',
 'Artois',
 'Conches',
 'hôtel de Nesle',
 'Rouées',
 'tour de Nesle',
 'Tour',
 'comté d’Artois',
 'Bourgogne',
 'Artois',
 'Artois',
 'Bourgogne',
 'Allemagne',
 'château de Conches',
 'Beaumont',
 'France',
 'Bourgogne',
 'Artois',
 'Artois',
 'Paris',
 'Artois']

In [73]:
ensemble_lieux = set(sorted(list_loc))
print(list(ensemble_lieux))

['Majorque', 'Brienne', 'Beauvais', 'Aucune', 'comté de Beaumont-le-Roger', 'Brésil', 'Neauphle', 'Bordeaux', 'Franche-Comté', 'Piacenza', 'comtes d’Artois', 'Sceaux', 'Cistercien', 'Vénétie', 'Forez', 'Jérusalem', 'Escot', 'Hongrie', 'Presles', 'ROUTE DE CLERMONT', 'Trésor', 'Oise', 'l’Espagne', 'Béthune', 'rue Saint-Merri', 'Boulogne', 'Hôtel', 'Middlesex', 'Montacute', 'Nogent-le-Roi', 'Car', 'Broderies', 'York', 'Petit-Pré-aux-Clercs', 'Mons-en-Pévèle', 'Rome', 'Chirk', 'Seigneur de Fontenay', 'Saint-Siège', 'Montaigu', 'Neauphle-le-Vieux', 'Milan', 'Évreux', 'Grand Nesle', 'Clermont', 'Arbois', 'Cornouailles', 'Mirepoix', 'ÉTAIT', 'Fréjus', 'VII', 'Vémars', 'comté de Flandre', 'Windsor', 'Achaïe', 'Notre', 'Beaune', 'Vélines', 'Toutes', 'hôtel d’Artois', 'Afrique', 'Grèce', 'rue aux Sorcières', 'Santa Maria dei Servi', 'Pensez', 'Petit Nesle', 'Montfaucon', 'Galerie mercière', 'Roncheville', 'Bigorre', 'Lorraine', 'Alips de Mons', 'archidiocèse de Sens', 'Tombeau du Christ', 'Nava

Une alternative est constituée par la librairie spacy, cf.: https://spacy.io/

## Quelques autre prétraitements

Autre prétraitement utile : remplacer certains motifs trouvés dans les textes.

In [50]:
print(X)
X_rep = [x.replace("is", "are") for x in X]
print(X_rep)

['Here is the adventures of an adventurous adventurer']
['Here are the adventures of an adventurous adventurer']


In [51]:
re.sub("[s|S]+\w*", "truc", X_rep[0])

'Here are the adventuretruc of an adventuroutruc adventurer'

Corriger automatiquement l'orthographe des mots grâce à la distance d'édition (ou distance Levenshtein, proposée en 1965).
Il s'agit de calculer le plus petit nombre d'opérations (insertion, suppression, remplacement) pour passer d'une chaîne à une autre.

In [52]:
# one possible implementation from https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [53]:
#levenshtein("chaîne", "chaine")
levenshtein("chaîne", "chiane")
#levenshtein("remerciement", "remerciments")

2